<div style="direction:rtl">
<h1>تولید کلمات کلیدی</h1>
</div>

<div style="direction:rtl"> 
    در این بخش به پیاده‌سازی کامل برنامه استخراج کلمات کلیدی از متن می‌پردازیم. <br>
    روند استخراج کلمات کلیدی را به صورت کلی بیان می‌کنیم: <br>
    <ol>
        <li>
            ابتدا متن ورودی نرمال سازی و توکن‌سازی شده و سایر کارهای مربوط به پیش‌پردازش اولیه روی آن انجام می‌شود.
        </li>
        <li>
            سپس متن کدگذاری شده و به برچسب‌گذار ادات سخن که در فایل دیگری از این پروژه آموزش داده شده‌است داده می‌شود و بر روی کلمات ورودی آن برچسب مربوط به نقش آن‌ها گذاشته می‌شود.
        </li>
        <li>
            سپس تکه (chunk) عبارت‌های معنی دار مانند عبارت‌های اسمی، فعلی و صفتی در دنباله کلمات ورودی با توجه به ادات سخنی که در بخش قبل به دست آمده‌است مشخص می‌شوند.
        </li>
        <li>
            اکنون دنباله‌هایی که حاوی عبارت‌های اسمی (زیرا کلمات کلیدی معمولاً اسم می‌باشند) هستند را میان تکه‌های به دست آمده جدا می‌کنیم. 
        </li>
        <li>
           سپس بر روی آن‌ها BPE را اجرا می‌کنیم و در صورتی که توسط این الگوریتم کلمه شکسته شد به این معناست که زیرکلمه‌ای از آن به شکل شایعی در کلمات کتاب آمده‌است و این به این معناست که احتمالاً کلمه کلیدی نیست. 
        </li>
        <li>
            و در نهایت کلمات شکسته نشده را فیلتر کرده و در صورتی که جزو کلمات نادرتر تشخیص داده شده نبودند به عنوان خروجی در نظر می‌گیریم.
        </li>
    </ol>
</div>

<div style="direction:rtl"> 
    <h2>بارگیری فایل‌های پیش‌پردازش و آماده‌سازی شده</h2>
</div>

In [ ]:
!pip install nltk

In [3]:
import nltk

nltk.download('punkt')

In [20]:
import json

with open('analysis.json') as fp:
    analysis_data = json.load(fp)

<div style="direction:rtl"> 
    <h3> بارگیری برچسب‌گذار ادات سخن</h3>
</div>

In [ ]:
!pip install tensorflow

In [4]:
with open('pos/words.txt') as fp:
    words_all = [line.strip() for line in fp.readlines()]
word2int = {k:i for i, k in enumerate(words_all)}
VOCAB_SIZE = len(word2int) + 1

In [5]:
with open('pos/tags.txt') as fp:
    tags_all = [line.strip() for line in fp.readlines()]
tag2int = {k:i for i, k in enumerate(tags_all)}
TAGS_NO = len(tag2int)

In [6]:
from tensorflow.keras.models import load_model
pos_tagger = load_model('models/pos_lstm_3.h5')

In [7]:
pos_tagger.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 50)]              0         
                                                                 
 embedding (Embedding)       (None, 50, 100)           7840700   
                                                                 
 bidirectional (Bidirectiona  (None, 50, 160)          115840    
 l)                                                              
                                                                 
 time_distributed (TimeDistr  (None, 50, 34)           5474      
 ibuted)                                                         
                                                                 
Total params: 7,962,014
Trainable params: 7,962,014
Non-trainable params: 0
_________________________________________________________________


<div style="direction:rtl"> 
    <h3> بارگیری تکه‌عبارت یاب (Phrase Chunker)</h3>
</div>

In [8]:
import pickle

with open('chunker.pkl', 'rb') as fp:
    chunk_parser = pickle.load(fp)

<div style="direction:rtl"> 
    <h3> بارگیری  BPE Encoder </h3>
</div>

In [9]:
with open('bpe-encoder.pkl', 'rb') as fp:
    bpe_encoder = pickle.load(fp)

<div style="direction:rtl"> 
    <h3> بارگیری کلمات نادرتر </h3>
</div>

In [70]:
with open('uncommon_words.txt') as fp:
    uncommon_words = set([line.strip() for line in fp.readlines()])

<div style="direction:rtl"> 
    <h2> پیش‌پردازش (نرمال‌سازی و توکن‌سازی) متن ورودی </h2>
</div>

In [10]:
import re


def encode_text(text):
    text = re.sub('[' + analysis_data['pun'] + '*]', '', text)
    text = re.sub('[!]', ' ', text)
    text = re.sub('ئ', 'ی', text)
    text = re.sub('ء', '', text)
    text = re.sub('[\s]+', ' ', text)
    tokens = nltk.tokenize.word_tokenize(text)
    return tokens, [word2int[word] if word in word2int else word2int['[UNK]'] for word in tokens]

<div style="direction:rtl"> 
    <h3> متن‌های ورودی</h3>
</div>

In [11]:
with open('kamal-al-din-sadoogh-clean.txt') as fp:
    lines = [line.strip() for line in fp.readlines()]

In [14]:
texts = lines[:10] # input texts

In [15]:
import numpy as np

<div style="direction:rtl"> 
    <h1> برچسب‌گذاری ادات سخن</h1>
</div>

In [16]:
SEQ_LEN = 50

In [17]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [18]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def pos_tag(text):
    test_X = []
    test_words = []

    text_words, text_tokens = encode_text(text)
    words_list = list(chunks(text_words, SEQ_LEN))
    tokens_list = list(chunks(text_tokens, SEQ_LEN))
    test_X += tokens_list
    test_words += words_list
    test_X = pad_sequences(test_X, maxlen=SEQ_LEN, padding='post')
    pred_outs = pos_tagger.predict(test_X)
    pred_args = np.argmax(pred_outs, axis=2)
    pred_tags = []
    for i, pred in enumerate(pred_args):
        cur_tags = [tags_all[i] if i in range(len(tags_all)) else 'UNK' for i in pred]
        cur_pairs = list(zip(test_words[i], cur_tags))
        pred_tags += cur_pairs
    return pred_tags

In [21]:
tagged_texts = []
for text in texts:
    tagged_texts.append(pos_tag(text))

In [22]:
tagged_texts

[[('ترجمه', 'N_SING'), ('کمال', 'N_SING'), ('الدین', 'N_SING')],
 [('مشخصات', 'N_PL'), ('کتاب', 'N_SING')],
 [('جلد', 'N_SING'), ('اول\u200c', 'FW')],
 [('[', 'DELM'), ('مقدمه', 'N_SING'), ('مؤلف', 'DELM'), (']', 'DELM')],
 [('اشاره', 'N_SING')],
 [('بسم', 'FW'), ('الله', 'FW'), ('الرحمن', 'FW'), ('الرحیم', 'FW')],
 [('(', 'DELM'),
  ('1', 'NUM'),
  (')', 'DELM'),
  ('حمد', 'N_SING'),
  ('از', 'P'),
  ('آن', 'PRO'),
  ('خداوندی', 'N_SING'),
  ('است', 'V_PRS'),
  ('که', 'CON'),
  ('پروردگار', 'N_SING'),
  ('جهانیان', 'N_PL'),
  ('است', 'V_PRS'),
  ('،', 'DELM'),
  ('و', 'CON'),
  ('درود', 'N_SING'),
  ('خداوند', 'N_SING'),
  ('بر', 'P'),
  ('پیامبرش', 'N_SING'),
  ('محمد', 'N_SING'),
  ('و', 'CON'),
  ('خاندان', 'N_SING'),
  ('طاهرینش', 'N_SING'),
  ('باد', 'V_SUB'),
  ('.', 'DELM')],
 [('حمد', 'N_SING'),
  ('سزاوار', 'ADJ'),
  ('خدای', 'N_SING'),
  ('یکتای', 'N_SING'),
  ('تنهاست', 'ADJ'),
  ('،', 'DELM'),
  ('بی\u200cنیاز', 'ADJ'),
  ('و', 'CON'),
  ('زنده', 'ADJ'),
  ('و', 'CON'),
  

<div style="direction:rtl"> 
    <h1> یافتن تکه‌عبارت‌ها (Phrase Chunking)</h1>
</div>

In [31]:
def phrase_chunk(tagged_text):
    tree = chunk_parser.parse(tagged_text)
    return tree

In [32]:
trees = []
for tagged_text in tagged_texts:
    trees.append(phrase_chunk(tagged_text))

<div style="direction:rtl"> 
    <h1> جدا کردن عبارت‌های اسمی و اسم‌های  موجود در آن‌ها</h1>
</div>

In [40]:
def extract_name_phrases(tree):
    subtrees = [t for t in list(tree.subtrees(filter=lambda x: x.label()=='NNP')) if isinstance(t[0], list)]
    word_sets = [set([d[0] for d in t.leaves()]) for t in subtrees]
    if not word_sets:
        return set()
    union_words = set.union(*word_sets)
    return union_words

In [41]:
extracted_names = []
for tree in trees:
    extracted_names.append(extract_name_phrases(tree))

<div style="direction:rtl"> 
    <h1> فیلتر کردن اسم‌های شکسته‌شده توسط BPE و اسم‌های نادرتر</h1>
</div>

In [71]:
def bpe_uncommon_filter(words):
    return uncommon_words.intersection(set([w for w in words if len(bpe_encoder.tokenize(w)) == 1]))

In [72]:
final_keywords = []
for names in extracted_names:
    final_keywords.append(bpe_uncommon_filter(names))

<div style="direction:rtl"> 
    <h1> کلمات کلیدی نهایی </h1>
</div>

In [73]:
final_keywords

[{'الدین', 'کمال'},
 set(),
 set(),
 set(),
 {'اشاره'},
 set(),
 {'جهانیان', 'حمد', 'خاندان', 'درود', 'پروردگار', 'پیامبرش'},
 {'ادراک',
  'اراده',
  'اکرام',
  'بلند',
  'توانا',
  'حمد',
  'حکیم',
  'دارای',
  'دیدگان',
  'زنده',
  'سزاوار',
  'صفات',
  'فضل',
  'مثلی',
  'مرتبه'},
 {'احسان',
  'تکلیف',
  'جاعل',
  'حکم',
  'خویشان',
  'زشتی',
  'ستم',
  'شهادت',
  'طاقت',
  'عدل',
  'عطا',
  'عملی',
  'فوق',
  'مالک',
  'معبودی'},
 {'بالغه', 'راست', 'سلامت'}]

<div style="direction:rtl">
<h1> پایان</h1>
</div>